In [85]:
#%%capture

#!pip install 'fhnw-nlp-utils>=0.8.0,<0.9.0'
#!pip install transformers
from fhnw.nlp.utils.processing import parallelize_dataframe
from fhnw.nlp.utils.processing import is_iterable
from fhnw.nlp.utils.storage import download
from fhnw.nlp.utils.storage import save_dataframe
from fhnw.nlp.utils.storage import load_dataframe
import pandas as pd
import numpy as np
import re
import tqdm
from datetime import datetime
#from transformers import AutoTokenizer, TFAutoModel
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import multilabel_confusion_matrix
import matplotlib.pyplot as plt
import tensorflow as tf
#from transformers import BertTokenizer, TFBertModel
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import re
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from fhnw.nlp.utils.preprocess import preprocess
tqdm.tqdm.pandas()
sns.set_style('whitegrid')

pd.options.display.max_colwidth = 600
pd.options.display.max_rows = 400


Laden der Daten

In [86]:
file = "data/german_news_articles_original_train.parq"
download("https://drive.switch.ch/index.php/s/mRnuzx4BLpMLqyz/download", file)
data_train = load_dataframe(file)

file = "data/german_news_articles_original_test.parq"
download("https://drive.switch.ch/index.php/s/DKUnZraeGp3EIK3/download", file)
data_test = load_dataframe(file)


data_train["split"] = "train"
data_test["split"] = "test"
data_all = pd.concat([data_train, data_test])
data = data_train

In [87]:
data_all = data_all.drop(["text"], axis=1, errors='ignore')

Encoden der 9 Labels

In [88]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label_code = le.fit(data_train['label'].drop_duplicates())
#y_train 

In [89]:
label = pd.DataFrame(data_train['label'].drop_duplicates(inplace=False))
label.reset_index(inplace = True, drop = True)
label

,label
0,Sport
1,Kultur
2,Web
3,Wirtschaft
4,Inland
5,Etat
6,International
7,Panorama
8,Wissenschaft


Vorbereiten der Daten und kurzer check der längen.

In [90]:
X_train = data['text_original']
X_test = data_test['text_original']
y_train = label_code.transform(data['label'])
y_true = label_code.transform(data_test['label'])
print(len(y_train),len(y_true), len(X_test))


9245 1028 1028


Der untere Code importiert das stopwords-Korpus aus dem Natural Language Toolkit (NLTK) und lädt die erforderlichen Ressourcen für deutsche Stoppwörter herunter. Anschließend erstellt er eine Menge deutscher Stoppwörter und speichert sie in der Variablen stopwords.

In [91]:
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

stopwords = set(stopwords.words("german"))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\reto.steiner\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\reto.steiner\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Dieser Code importiert die xgboost-Bibliothek und erstellt ein Objekt der Klasse xgb.XGBClassifier. Das Objekt clf wird verwendet, um ein XGBoost-Modell zu trainieren und zu verwenden.

Die Parameter des Objekts clf sind wie folgt:

objective='multi:softmax': Dieser Parameter legt die Zielfunktion des Modells fest. In diesem Fall wird eine harte Klassifikation mit 9 Klassen verwendet.
num_class=9: Dieser Parameter legt die Anzahl der Klassen fest, die das Modell vorhersagen soll.
missing=0: Dieser Parameter legt den Wert fest, der als fehlend behandelt wird. In diesem Fall wird der Wert 0 verwendet.
early_stopping_rounds=10: Dieser Parameter legt die Anzahl der Runden fest, nach denen das Training gestoppt wird, wenn sich die Leistung auf einem Validierungsdatensatz nicht mehr verbessert.
seed=42: Dieser Parameter legt den Zufallssamen für das Training des Modells fest. Dies kann hilfreich sein, um die Reproduzierbarkeit des Modells zu verbessern.

In [92]:
import xgboost as xgb
clf = xgb.XGBClassifier(
    objective='multi:softmax',  # Für eine harte Klassifikation
    num_class=9,  # Anzahl der Klassen
    missing=0,  # Wert, der als fehlend behandelt wird
    #early_stopping_rounds=10,  # Frühes Stoppen nach 10 Runden ohne Verbesserung  # Zu überwachende Metriken
    seed=42  # Zufallssamen für die Wiederholbarkeit
)


Dieser Code transformiert die Trainingsdaten und Testdaten in TF-IDF-Vektoren. Die Methode fit_transform() passt das TfidfVectorizer-Objekt an die Trainingsdaten an und transformiert dann sowohl die Trainingsdaten als auch die Testdaten in TF-IDF-Vektoren.

In [93]:
def analyze(text):
    return preprocess(text, stopwords=stopwords, do_compound_word_split=True)

Vec = TfidfVectorizer(analyzer=analyze)
X_train_vec = Vec.fit_transform(X_train)
X_test_vec = Vec.fit_transform(X_test)

Model mit Smote trainieren,vorhersagen und bewerten

In [95]:
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE


# Create the pipeline
pipeline = make_pipeline(TfidfVectorizer(analyzer=analyze),
    SMOTE(random_state=12),
    clf
)

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer=<function analyze at 0x000002115A897B80>)),
                ('smote', SMOTE(random_state=12)),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               featur...
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=0,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_class=9, num_parallel_tree=None,
                               objective='multi:softmax', ...))])

In [96]:
y_pred = pipeline.predict(X_test)


In [97]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score ,precision_score, make_scorer, recall_score

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred,average='macro')
#roc_auc = roc_auc_score(y_true, y_pred, average='ovs')
prec = precision_score(y_true, y_pred,average='macro')
rec = recall_score(y_true, y_pred,average='macro')
print('Accuracy:', acc)
print('F1-Score:', f1)
#print('ROC-AUC:', roc_auc)
print('Recall:', rec)
print('Precision:', prec)

Accuracy: 0.8249027237354085
F1-Score: 0.8276742793599102
Recall: 0.8236759163908027
Precision: 0.8336218281598674


In [115]:
conf = multilabel_confusion_matrix(y_true, y_pred)

Das gleiche ohne Smote und vergleichen

In [99]:


pipeline_withot_smote = Pipeline([
         ('vec', TfidfVectorizer(analyzer=analyze)),
         ("clf", clf)
        ])

pipeline_withot_smote.fit(X_train, y_train)

Pipeline(steps=[('vec',
                 TfidfVectorizer(analyzer=<function analyze at 0x000002115A897B80>)),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=0,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_class=9, num_parallel_tree=None,
                               objective='multi:softmax', ...))])

In [100]:
y_pred = pipeline_withot_smote.predict(X_test)


In folgenden Metriken kann gesehen werden, das die Klassifizierung ohne SMOTE tendenziell besser ist

In [101]:
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred,average='macro')
#roc_auc = roc_auc_score(y_true, y_pred, average='ovs')
prec = precision_score(y_true, y_pred,average='macro')
rec = recall_score(y_true, y_pred,average='macro')
print('Accuracy:', acc)
print('F1-Score:', f1)
#print('ROC-AUC:', roc_auc)
print('Recall:', rec)
print('Precision:', prec)

Accuracy: 0.8317120622568094
F1-Score: 0.8337614652957991
Recall: 0.8291292937022335
Precision: 0.8419076583520317


Folgend noch der Vergleich der Confusion Matrizen mit und ohne SMOTE. Die Determinante der Differenzen der Matrixen muss 0 ergeben, dies ist bei jedem Label der Fall.
Ein positives Ergebnis in der ersten Spalte bedeutet in der Folgenden Auswertung das mit SMOTE eine besser Klassifizierung stattfindet als ohne.
Somit bestätigt das ergebnis die anderen Metriken, nur bei 2 Labels ist das Ergebnis mit SMOTE besser.
Daher wird folgend ohne künstlich erzeugte Daten weitergearbeitet. Was eine mögliche Verbesserung bringen könnte, wäre Data Agumentation. Hier ein Artikel https://neptune.ai/blog/data-augmentation-nlp

In [114]:
conf_withot_smote = multilabel_confusion_matrix(y_true, y_pred)
conf1 = conf - conf_withot_smote

for i in conf1:
    print(i[0,0] + i[1,1],i[1,0] + i[0,1])
    


-1 1
-10 10
-4 4
-2 2
-4 4
4 -4
2 -2
-1 1
2 -2
